In [110]:
from pynq import Overlay, Xlnk, MMIO, Interrupt, GPIO
import pynq.lib.dma
import numpy as np
from pynq import allocate
import asyncio
# PUF_debug.bit is the basic Arbiter PUF
puf = Overlay("./app/puf_axi/PUF_XOR7_stable.bit")
# puf = Overlay("./app/puf_axi/PUF_XOR2.bit")
# puf = Overlay("./app/puf_axi/PUF_1020.bit")
puf.download()
PUF = puf.pl_ram_ctrl_0
bram = puf.axi_bram_ctrl_0
def concat_uint32_to_uint64(uint32_1, uint32_2):
    # 将第一个uint32左移32位，然后与第二个uint32进行按位或运算
    uint64_result = (uint32_1 << 32) | uint32_2
    return np.uint64(uint64_result)
Len = 103
BRAM_BYTENUM = 4
len = np.uint32(Len*BRAM_BYTENUM)
valid_L = 46
# intr = puf.axi_intc_0
# gpio = puf.axi_gpio_0

In [72]:
puf.ip_dict

{'axi_bram_ctrl_0': {'addr_range': 4096,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb0288990>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_bram_ctrl_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S_AXI',
  'parameters': {'BMG_INSTANCE': 'EXTERNAL',
   'C_BRAM_ADDR_WIDTH': '10',
   'C_BRAM_INST_MODE': 'EXTERNAL',
   'C_ECC': '0',
   'C_ECC_ONOFF_RESET_VALUE': '0',
   'C_ECC_TYPE': '0',
   'C_FAMILY': 'zynq',
   'C_FAULT_INJECT': '0',
   'C_MEMORY_DEPTH': '1024',
   'C_RD_CMD_OPTIMIZATION': '0',
   'C_READ_LATENCY': '1',
   'C_SINGLE_PORT_BRAM': '1',
   'C_S_AXI_ADDR_WIDTH': '12',
   'C_S_AXI_BASEADDR': '0x40000000',
   'C_S_AXI_CTRL_ADDR_WIDTH': '32',
   'C_S_AXI_CTRL_DATA_WIDTH': '32',
   'C_S_AXI_DATA_WIDTH': '32',
   'C_S_AXI_HIGHADDR': '0x40000FFF',
   'C_S_AXI_ID_WIDTH': '1',
   'C_S_AXI_PROTOCOL': 'AXI4',
   'C_S_AXI_SUPPORTS_NARROW_BURST': '0',
   'Component_Name': 'top_axi_bram_ctrl_0_0',
   'DATA_WIDTH': '32',
   'ECC_ONOFF_RESET_VALUE': '0',
   'ECC_TYPE

In [122]:
challenges = np.random.randint(0,2**32,size=Len,dtype=np.uint32)

In [118]:
start = np.random.randint(0,2**32,dtype=np.uint32)
start = 2**7
challenges = np.arange(start,start+Len,1,dtype=np.uint32)

In [119]:
for i in range(0,50,1):
    challenges[i] = 0

In [120]:
challenges

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 178, 179,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
       193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205,
       206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
       219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230], dtype=uint32)

In [124]:
import time
N_total = 200000
responses = np.zeros(N_total,dtype=np.int)
batch = valid_L
runs = N_total // batch
i=0
with open(f'7puf_output2_{N_total}.txt', 'w') as file:
    for r in range(runs):
        challenges = np.random.randint(0,2**32,size=Len,dtype=np.uint32)

        counter = 0
        for i in range(0,BRAM_BYTENUM*Len,BRAM_BYTENUM):
            bram.write(i,challenges[counter].tobytes())
        #     bram.write(i,test_data[counter].tobytes())
            counter += 1

        PUF.write(8,len.tobytes())
        PUF.write(12,0x0)
        PUF.write(4,0x1)
        PUF.write(0,0x1)

        while(1):
            flag = True
            for i in range(BRAM_BYTENUM*30,BRAM_BYTENUM*40,BRAM_BYTENUM):
                if (bram.read(i)>=2**7):
#                     print(bram.read(i))
                    flag = False
                    break
            if(flag):
                break

#         time.sleep(2)

        response = []
        count=0
        for i in range(BRAM_BYTENUM*0,BRAM_BYTENUM*50,BRAM_BYTENUM):
            response.append(bram.read(i))
        #     print(f"{count}-th responses:",bram.read(i))
        #     print(f"{count}-th challenges:",challenges[count+50]%2)
            count+=1

        valid = np.array(response[4:50]).astype(int)
#         valid = np.array(response[4:102]).astype(int)
        # gpio.ip2intc_irpt.wait()

        # gpio_intr.wait()

        for i, r in enumerate(valid):
        #     print(f"{challenges[i]:08X}")
            c = concat_uint32_to_uint64(challenges[i],challenges[i+50])
#             c = challenges[i]
#             print(f"{i}-th challenge={c:016X}, response={r:07b}")
            file.write(f'{c:016X};{r:07b}\n')
    print("Done~")

Done~
